# Estratégias de treino

Até agora no curso, vimos somente uma estratégia de treino para redes neurais: o treinamento do zero.
Entretanto, há outras formas de se explorar redes neurais.
Nessa aula, vamos rever a estratégia treinamento do zero além de apresentar duas novas formas:

1.   rede neural como um extrator de características, e
2.   *fine-tuning*.

Para cada uma dessas estratégias, vamos apresentar sua definição, vantagens e desvantagens.
Antes, vamos instalar o MXNet, importar alguns pacotes e definir funções para carregar os dados.


In [0]:
!pip install mxnet-cu100

# imports basicos
import time, os, sys, numpy as np
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'mnist')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.MNIST(root=root, train=True)
    mnist_test = gdata.vision.MNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do CIFAR 10
# https://www.cs.toronto.edu/~kriz/cifar.html
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'cifar10')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    cifar10_train = gdata.vision.CIFAR10(root=root, train=True)
    cifar10_test = gdata.vision.CIFAR10(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(cifar10_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(cifar10_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter
  
# funções básicas
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

     |████████████████████████████████| 488.3MB 34kB/s 
     |████████████████████████████████| 13.8MB 35.1MB/s 
ERROR: spacy 2.1.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.54 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


## Treinamento do zero

Como dito anteriormente, essa foi a única estratégia vista até o momento no curso.
Nessa estratégia, uma rede neural é proposta, **inicializada com pesos aleatórios** e treinada até convergir.
A **vantagem** dessa estratégia é liberdade para definir como quiser a arquitetura da rede e seus hiper-parâmetros
Por outro lado, a **desvantagem** é que essa estratégia requer muitos dados para convergir a rede inicializada aleatoriamente.
Logo, se tivermos poucos dados, essa não é a estratégia mais recomendada.
Abaixo, uma representação visual dessa estratégia.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1_bBQjyoDqB3kQMncmVkuJwSxDs3rqUmM">
</p>

Apesar de já termos visto essa estratégia na prática, vamos vê-la aqui novamente para efeitos de comparação com as outras técnicas. Para tal, vamos, primeiro, definimos a arquitetura da [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf).



In [0]:
class AlexNet(nn.HybridBlock):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    classes : int, default 10
        Number of classes for the output layer.
    """
    def __init__(self, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        with self.name_scope():
            self.features = nn.HybridSequential(prefix='')
            with self.features.name_scope():
                self.features.add(nn.Conv2D(64, kernel_size=11, strides=4, padding=0, activation='relu'))  # entrada: (b, 3, 227, 227) e saida: (b, 64, 55, 55)
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))                                    # entrada: (b, 64, 55, 55) e saida: (b, 64, 27, 27)
                self.features.add(nn.Conv2D(192, kernel_size=5, padding=2, activation='relu'))             # entrada: (b, 64, 27, 27) e saida: (b, 192, 27, 27)
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))                                    # entrada: (b, 192, 27, 27) e saida: (b, 192, 13, 13)
                self.features.add(nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'))             # entrada: (b, 192, 13, 13) e saida: (b, 384, 13, 13)
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'))             # entrada: (b, 384, 13, 13) e saida: (b, 256, 13, 13)
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'))             # entrada: (b, 256, 13, 13) e saida: (b, 256, 13, 13)
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))                                    # entrada: (b, 256, 13, 13) e saida: (b, 256, 6, 6)
                self.features.add(nn.Flatten())                                                            # entrada: (b, 256, 13, 13) e saida: (b, 256*6*6) = (b, 9216)
                self.features.add(nn.Dense(4096, activation='relu'))                                       # entrada: (b, 9216) e saida: (b, 4096)
                self.features.add(nn.Dropout(0.5))
                self.features.add(nn.Dense(4096, activation='relu'))                                       # entrada: (b, 4096) e saida: (b, 4096)
                self.features.add(nn.Dropout(0.5))

            self.output = nn.Dense(classes)  # entrada: (b, 4096) e saida: (b, 10)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x

Nesse bloco abaixo, carregamos o dataset [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) para treinar a rede.
Notem que a rede é inicializada aleatoriamente na linha `net.initialize(init.Normal(sigma=0.01), ctx=ctx)`.
Logo, não temos controle sobre esses pesos que podem assumir qualquer valor.

In [0]:
NUM_CLASSES = 10

num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
net = AlexNet(NUM_CLASSES)
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd_lambda, 'momentum': 0.9})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 2.3025, train acc 0.100, test loss 2.3024, test acc 0.100, time 69.7 sec
epoch 2, train loss 2.3023, train acc 0.110, test loss 2.3020, test acc 0.178, time 66.6 sec
epoch 3, train loss 2.3019, train acc 0.149, test loss 2.3014, test acc 0.183, time 66.7 sec
epoch 4, train loss 2.3011, train acc 0.158, test loss 2.3001, test acc 0.226, time 66.7 sec
epoch 5, train loss 2.2992, train acc 0.189, test loss 2.2961, test acc 0.199, time 67.0 sec
epoch 6, train loss 2.2820, train acc 0.169, test loss 2.2130, test acc 0.167, time 66.8 sec
epoch 7, train loss 2.1272, train acc 0.211, test loss 1.9905, test acc 0.264, time 66.8 sec
epoch 8, train loss 1.9652, train acc 0.272, test loss 1.8956, test acc 0.307, time 66.7 sec
epoch 9, train loss 1.8751, train acc 0.306, test loss 1.8088, test acc 0.346, time 66.8 sec
epoch 10, train loss 1.7789, train acc 0.342, test loss 1.7553, test acc 0.360, time 67.2 sec
epoch 11, train loss 1.6997, train acc 0.375, test

É muito comum se usar redes já existentes para aprender características em novos dados.
Por isso, muitos frameworks já deixam as arquiteturas mais famosas pré-implementadas para que possam ser usadas.

No MXNet, podemos importar uma rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) usando o pacote [model_zoo](https://mxnet.incubator.apache.org/api/python/gluon/model_zoo.html) do MXNet.
Há várias arquiteturas pré-definidas nessa biblioteca, incluindo várias [DenseNets](https://arxiv.org/pdf/1608.06993.pdf) e [ResNets](https://arxiv.org/abs/1603.05027), [VGGs](https://arxiv.org/abs/1409.1556), [SqueezeNets](https://arxiv.org/abs/1602.07360), etc.

In [0]:
from mxnet.gluon.model_zoo import vision

NUM_CLASSES = 10
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
net = vision.alexnet(classes=NUM_CLASSES)  # http://mxnet.incubator.apache.org/versions/master/tutorials/gluon/pretrained_models.html
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd_lambda, 'momentum': 0.9})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 2.3026, train acc 0.101, test loss 2.3024, test acc 0.109, time 70.5 sec
epoch 2, train loss 2.3023, train acc 0.121, test loss 2.3021, test acc 0.186, time 67.2 sec
epoch 3, train loss 2.3020, train acc 0.134, test loss 2.3016, test acc 0.220, time 67.2 sec
epoch 4, train loss 2.3013, train acc 0.161, test loss 2.3004, test acc 0.192, time 67.1 sec
epoch 5, train loss 2.2997, train acc 0.189, test loss 2.2973, test acc 0.216, time 67.3 sec
epoch 6, train loss 2.2921, train acc 0.178, test loss 2.2704, test acc 0.142, time 67.2 sec
epoch 7, train loss 2.1805, train acc 0.189, test loss 2.0912, test acc 0.237, time 67.5 sec
epoch 8, train loss 2.0156, train acc 0.259, test loss 1.9899, test acc 0.285, time 67.2 sec
epoch 9, train loss 1.9183, train acc 0.292, test loss 1.8192, test acc 0.330, time 67.0 sec
epoch 10, train loss 1.8218, train acc 0.323, test loss 1.7433, test acc 0.343, time 67.6 sec
epoch 11, train loss 1.7511, train acc 0.352, test

## Extrator de características

A terceira e última estratégia, mostrada na figura abaixo, é usar uma rede neural pré-treinada em algum dataset grande para extrair características de um outro dataset. Essa estratégia é preferível quando o dataset que se quer extrair as *features* tem muito poucas amostras, inviabilizando o treinamento ou *fine-tuning* da rede.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1pWGfQIAeOODIvm-IQ7De4kl60XpRYbb5">
</p>

Existem duas formas de se explorar essa estratégia. A primeira consiste em substituir e treinar somente a última camada da rede neural. Nessa primeira forma, todas as outras camadas da rede ficam com *learning rate* 0, ou seja, não aprendem nada, e são somente usadas como codificadores/extratores de características. A segunda forma, *features* das imagens do dataset que se quer classificar são extraídas da penúltima camada da rede pré-treinada (geralmente, a camada antes da camada de classificação). Essas *features* são então usadas para se treinar um agoritmo externo (como um SVM ou *random forest*), que então classifica o dataset.

In [0]:
from mxnet.gluon.model_zoo import vision

NUM_CLASSES = 10
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
net = vision.alexnet(pretrained=True, ctx=ctx)
print(net)
with net.name_scope():
    net.output = gluon.nn.Dense(NUM_CLASSES)  # substitui a ultima camada por uma nova com 10 classes
    net.output.initialize(init.Normal(sigma=0.01), ctx=ctx)  # inicializa essa camada aleatoriamente de acordo com uma distribuicao Normal
print(net)

# este loop abaixo ajusta o multiplicador do learning rate para as camadas
for key, value in net.collect_params().items():
    print(key, value.lr_mult)
    if 'dense3' not in key:  # essa eh a nota camada e nao vamos mudar o multiplicador do seu learning rate
      value.lr_mult = 0.0  # so aprenderemos a ultima camada
    print(key, value.lr_mult)
    
# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd_lambda, 'momentum': 0.9})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

AlexNet(
  (features): HybridSequential(
    (0): Conv2D(3 -> 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2), Activation(relu))
    (1): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (2): Conv2D(64 -> 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), Activation(relu))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (4): Conv2D(192 -> 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), Activation(relu))
    (5): Conv2D(384 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), Activation(relu))
    (6): Conv2D(256 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), Activation(relu))
    (7): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (8): Flatten
    (9): Dense(9216 -> 4096, Activation(relu))
    (10): Dropout(p = 0.5, axes=())
    (11): Dense(4096 -> 4096, Activation(relu))
    (12): Dropout(p = 0.5, axes=())
  )
  (output): Dense(4096 -> 1000, linear)
)
AlexNe

In [0]:
from mxnet.gluon.model_zoo import vision

NUM_CLASSES = 10
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
net = vision.alexnet(pretrained=True, ctx=ctx)
    
# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)  

first = True
for batch in train_iter:
    features, labels, _ = _get_batch(batch, [ctx])
    for X, y in zip(features, labels):
        y = y.astype('float32')
        features = net.features(X)
        if first is True:
          train_features = features.asnumpy()
          train_labels = y.asnumpy()
          first = False
        else:
          train_features = np.concatenate((train_features, features.asnumpy()))
          train_labels = np.concatenate((train_labels, y.asnumpy()))
          
          
first = True
for batch in test_iter:
    features, labels, _ = _get_batch(batch, [ctx])
    for X, y in zip(features, labels):
        y = y.astype('float32')
        features = net.features(X)
        if first is True:
          test_features = features.asnumpy()
          test_labels = y.asnumpy()
          first = False
        else:
          test_features = np.concatenate((test_features, features.asnumpy()))
          test_labels = np.concatenate((test_labels, y.asnumpy()))
          
print(train_features.shape, test_features.shape)


(50000, 4096) (10000, 4096)


In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

clf = LinearSVC()
clf.fit(train_features, train_labels)

pred = clf.predict(test_features)
print(accuracy_score(test_labels, pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7043


## *Fine-tuning*

A segunda estratégia é chamada de *fine-tuning*, e é comumente classificada como um estratégia de *transfer learning*, onde o aprendizado é transferido entre datasets.
Especificamente, esta estratégia, representada na figura abaixo, tenta usar um modelo pré-treinado aprendido anteriormente em algum dataset (geralmente muito grande, como o [ImageNet](http://www.image-net.org/)) para classificar outro conjunto de dados diferentes (geralmente com poucas amostras).

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1CoOfpMcQAEl9YAL0lgW11LLYpDcnL4dQ">
</p>

Como esses dados podem possuir características diferentes, treinamos a rede usando um *learning rate* pequeno, apenas para fazer pequenos ajustes nos pesos. Entretanto, como esses datasets geralmente tem número e classes diferentes, a última camada não é usada nessa transferência de peso e, geralmente, é inicializada aleatoriamente (e por isso, tem um *learning rate* mais alto que as demais camadas).

Por fim, é um [fato conhecido](https://arxiv.org/pdf/1602.01517.pdf) que as redes neurais conseguem aprender características de baixo nível nas camadas iniciais. Geralmente, essas características são comuns à vários datasets. Por isso, uma opção durante o processo de *fine-tuning* é "congelar" as camadas iniciais (ou seja, não treiná-las) e treinar somente as demais camadas com taxa de aprendizado bem pequeno (exceto pela camada de classificação).

No bloco de código abaixo, importamos a rede pré-treinada [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), que foi treinada no dataset do [ImageNet](http://www.image-net.org/), que tem 1000 classes. Como iremos fazer *fine-tuning* nessa arquitetura para o dataset do [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), que tem somente 10 classes, removeremos a última camada e criaremos uma nova camada inicializada aleatoriamente.

In [0]:
from mxnet.gluon.model_zoo import vision

NUM_CLASSES = 10
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
net = vision.alexnet(pretrained=True, ctx=ctx)
print(net)
with net.name_scope():
    net.output = gluon.nn.Dense(NUM_CLASSES)  # substitui a ultima camada por uma nova com 10 classes
    net.output.initialize(init.Normal(sigma=0.01), ctx=ctx)  # inicializa essa camada aleatoriamente de acordo com uma distribuicao Normal
print(net)

# este loop abaixo ajusta o multiplicador do learning rate para as camadas
for key, value in net.collect_params().items():
    print(key, value.lr_mult)
    if 'dense3' not in key:  # essa eh a nota camada e nao vamos mudar o multiplicador do seu learning rate
      value.lr_mult = 0.1
    print(key, value.lr_mult)
    
# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd_lambda, 'momentum': 0.9})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

AlexNet(
  (features): HybridSequential(
    (0): Conv2D(3 -> 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2), Activation(relu))
    (1): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (2): Conv2D(64 -> 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), Activation(relu))
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (4): Conv2D(192 -> 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), Activation(relu))
    (5): Conv2D(384 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), Activation(relu))
    (6): Conv2D(256 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), Activation(relu))
    (7): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (8): Flatten
    (9): Dense(9216 -> 4096, Activation(relu))
    (10): Dropout(p = 0.5, axes=())
    (11): Dense(4096 -> 4096, Activation(relu))
    (12): Dropout(p = 0.5, axes=())
  )
  (output): Dense(4096 -> 1000, linear)
)
AlexNe

## Prática

1. É possível melhorar o resultado obtido anteriormente?
Estude o [model_zoo](https://mxnet.incubator.apache.org/api/python/gluon/model_zoo.html)  e tente usar as estratégias anteriores com diferentes redes neurais para melhorar o resultado.
Algumas redes possíveis:

- [MobileNets](https://arxiv.org/abs/1801.04381)
- [VGGs](https://arxiv.org/abs/1409.1556)
- [ResNets](https://arxiv.org/abs/1603.05027)
- [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

2. Procure agora congelar algumas camadas para realizar o *fine-tuning*. Essa estratégia é melhor quando se tem poucas imagens para fazer o *fine-tuning*.

3. Procura usar outros algoritmos de aprendizado de máquina (como [*random forest*](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) e [SVM-RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)) para classificar *deep features* extraídas de uma rede neural pré-treinada.
  1. Procure também extrair e classificar *features* de outras camadas convolucionais.

4. Procure usar as diferentes estratégias para melhorar os resultados dos datasets que já usamos, como [MNIST](https://mxnet.incubator.apache.org/api/python/gluon/data.html#module-mxnet.gluon.data.vision) e [Fashion MNIST](https://mxnet.incubator.apache.org/api/python/gluon/data.html#mxnet.gluon.data.vision.datasets.FashionMNIST).